In [29]:
import pandas as pd
import numpy as np
from sklearn.naive_bayes import GaussianNB, CategoricalNB
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score

In [30]:
data = pd.read_csv("../dataset")
x_columns = data.columns.drop('label')
x = data[x_columns]
x = x.values
y = data['label']
le_proto = LabelEncoder()
x[:, 0] = le_proto.fit_transform(x[:, 0])
# le_label = LabelEncoder()
# y = le_label.fit_transform(y)
# sc = StandardScaler()
# x=sc.fit_transform(x)

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=0)

In [31]:
gnb = GaussianNB()

gnb.fit(x, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [32]:
datatest = pd.read_csv("../datatest")
x_columns = datatest.columns.drop('label')
x = datatest[x_columns]
x = x.values
y = datatest['label']

x[:, 0] = le_proto.transform(x[:, 0])
#joblib.dump(le_proto, '../preproc/le_proto2.joblib',protocol=1) #save label encoder weights to file
# y = le_label.fit_transform(y)
#joblib.dump(le_label, '../preproc/le_label2.joblib',protocol=1) #save label encoder weights to file
# x=sc.transform(x)
#joblib.dump(sc, '../preproc/scaler2.joblib',protocol=1)
# x = np.expand_dims(x, axis=2)
data = np.array([['icmp',0.31,98,1.0,9,1,1,1.0]])
data[:, 0] = le_proto.transform(data[:, 0])
data = data.astype(np.float64)
y_pred = gnb.predict(data)
# y_pred = le_label.inverse_transform(y_pred)


In [5]:
stats = {}
labels = ['normal','tcpsynflood','icmpflood','udpflood','ipsweep','portscan','pingofdeath']
for i in labels:
    stats[i]={}
    for j in labels:
        stats[i][j] = 0

In [6]:
for i in range(len(y)):
    stats[y[i]][y_pred[i]] += 1
stats

{'normal': {'normal': 751,
  'tcpsynflood': 0,
  'icmpflood': 21,
  'udpflood': 0,
  'ipsweep': 90,
  'portscan': 376,
  'pingofdeath': 4},
 'tcpsynflood': {'normal': 0,
  'tcpsynflood': 1045,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'icmpflood': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 784,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'udpflood': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 821,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 0},
 'ipsweep': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 647,
  'portscan': 0,
  'pingofdeath': 0},
 'portscan': {'normal': 3,
  'tcpsynflood': 103,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 812,
  'pingofdeath': 0},
 'pingofdeath': {'normal': 0,
  'tcpsynflood': 0,
  'icmpflood': 0,
  'udpflood': 0,
  'ipsweep': 0,
  'portscan': 0,
  'pingofdeath': 461}}

In [7]:
tpr={}
fpr={}
precision={}
f1score={}
for i in labels:
    tp = stats[i][i]
    fp = 0
    tn = 0
    fn = 0
    for j in labels:
        if i != j:
            fn += stats[i][j]
            fp += stats[j][i]
            for k in stats[j].keys():
                if k != i:
                    tn += stats[j][k]
    tpr[i] = tp / (tp + fn)
    fpr[i] = fp / (fp + tn)
    precision[i] = tp / (tp + fp)
    f1score[i] = 2/(1/tpr[i] + 1/precision[i])
print('TPR: \n ' + str(tpr))
print('FRP:\n ' + str(fpr))
# print('Precision:\n ' + str(precision))
print('F1-score: \n' + str(f1score))

print("Accuracy:",metrics.accuracy_score(y, y_pred))

TPR: 
 {'normal': 0.604669887278583, 'tcpsynflood': 1.0, 'icmpflood': 1.0, 'udpflood': 1.0, 'ipsweep': 1.0, 'portscan': 0.8845315904139434, 'pingofdeath': 1.0}
FRP:
 {'normal': 0.0006415739948674081, 'tcpsynflood': 0.02113687666735071, 'icmpflood': 0.004090377873003506, 'udpflood': 0.0, 'ipsweep': 0.01707455890722823, 'portscan': 0.0752, 'pingofdeath': 0.0007330034817665384}
F1-score: 
{'normal': 0.75250501002004, 'tcpsynflood': 0.9530323757409942, 'icmpflood': 0.9867841409691629, 'udpflood': 1.0, 'ipsweep': 0.934971098265896, 'portscan': 0.7711301044634378, 'pingofdeath': 0.9956803455723543}
Accuracy: 0.8991213247718824
